In [1]:
import underworld as uw
from underworld import function as fn
import glucifer
import numpy as np
import math

In [2]:
from planetengine.utilities import quickShow

In [3]:
mesh = uw.mesh.FeMesh_Annulus(
    elementRes = (32, 32),
    radialLengths = (1., 2.),
    angularExtent = (60., 120.)
    )

In [4]:
temperatureField = uw.mesh.MeshVariable(mesh, 1)
temperatureDotField = uw.mesh.MeshVariable(mesh, 1)
pressureField = uw.mesh.MeshVariable(mesh.subMesh, 1)
velocityField = uw.mesh.MeshVariable(mesh, 2)

In [5]:
inner = mesh.specialSets["inner"]
outer = mesh.specialSets["outer"]
sides = mesh.specialSets["MaxJ_VertexSet"] + mesh.specialSets["MinJ_VertexSet"]

velBC = uw.conditions.RotatedDirichletCondition(
    variable = velocityField,
    indexSetsPerDof= (inner + outer, sides),
    basis_vectors = (mesh.bnd_vec_normal, mesh.bnd_vec_tangent)
    )

tempBC = uw.conditions.DirichletCondition(
    variable = temperatureField,
    indexSetsPerDof = (inner + outer,)
    )

In [6]:
vc = uw.mesh.MeshVariable(mesh = mesh, nodeDofCount = 2)
vc_eqNum = uw.systems.sle.EqNumber(vc, False )
vcVec = uw.systems.sle.SolutionVector(vc, vc_eqNum)

In [7]:
isoviscous = fn.misc.constant(True)

In [8]:
invDensityFn = temperatureField * 1e7
buoyancyFn = invDensityFn * mesh.unitvec_r_Fn
magnitude = fn.math.sqrt(fn.coord()[0]**2 + fn.coord()[1]**2)
depthFn = mesh.radialLengths[1] - magnitude
yieldStressFn = 4e5 + (1e7 * depthFn)
secInvFn = fn.tensor.second_invariant(fn.tensor.symmetric(vc.fn_gradient))
plasticViscFn = yieldStressFn / (2. * secInvFn + 1e-18)
creepViscFn = fn.math.pow(fn.misc.constant(3e4), -1. * (temperatureField - 1.))
viscoplastic = fn.misc.min(3e4, fn.misc.max(1., fn.misc.min(creepViscFn, plasticViscFn)))

viscosityFn = fn.branching.conditional([(isoviscous, 1.), (True, viscoplastic)])

stokes = uw.systems.Stokes(
    velocityField = velocityField,
    pressureField = pressureField,
    conditions = [velBC,],
    fn_viscosity = viscosityFn,
    fn_bodyforce = buoyancyFn,
    _removeBCs = False,
    )

solver = uw.systems.Solver(stokes)

In [9]:
def postSolve():
    # realign solution using the rotation matrix on stokes
    uw.libUnderworld.Underworld.AXequalsY(
        stokes._rot._cself,
        stokes._velocitySol._cself,
        vcVec._cself,
        False
        )
    # remove null space - the solid body rotation velocity contribution
    uw.libUnderworld.StgFEM.SolutionVector_RemoveVectorSpace(
        stokes._velocitySol._cself, 
        stokes._vnsVec._cself
        )

In [80]:
def solve():
    velocityField.data[:] = 0.
    solver.solve(
        nonLinearIterate = not isoviscous.value,
        callback_post_solve = postSolve
        )
    uw.libUnderworld.Underworld.AXequalsX(
        stokes._rot._cself,
        stokes._velocitySol._cself,
        False
        )

In [81]:
def initialise():
    function = (2. - fn.input()[0]) + (fn.math.cos(2. - fn.input()[0]) * fn.math.sin(.5 * fn.input()[1] / 180.))
    temperatureField.data[:] = function.evaluate(mesh.radialData)
    temperatureField.data[outer] = 0.
    temperatureField.data[inner] = 1.
    velocityField.data[:] = [0., 0.]
    pressureField.data[:] = 0.
    temperatureDotField.data[:] = 0.

In [82]:
def build_int():
    surfIntegral = uw.utils.Integral(
        fn.math.dot(velocityField, velocityField), mesh,
        integrationType = 'surface',
        surfaceIndexSet = outer,
        )
    return surfIntegral

In [83]:
def test(isovisc):
    isoviscous.value = isovisc
    print("Isoviscous: ", isovisc)
    initialise()
    solve()
    surfIntegral = build_int()
    velCopy = velocityField.data.copy()
    tempCopy = temperatureField.data.copy()
    pressCopy = pressureField.data.copy()
    tempdotCopy = temperatureDotField.data.copy()
    for _ in range(10):
        solve()
        surfVal = surfIntegral.evaluate()[0]
        print(surfVal)
        check = all([
            np.allclose(velCopy, velocityField.data),
            np.allclose(tempCopy, temperatureField.data),
            np.allclose(pressCopy, pressureField.data),
            np.allclose(tempdotCopy, tempdotCopy.data),
            ])
        print(check)

In [93]:
def test2(isovisc):
    isoviscous.value = isovisc
    print("Isoviscous: ", isovisc)
    initialise()
    solve()
    velCopy = velocityField.data.copy()
    solve()
    print("Vel field unchanged: ", np.allclose(velCopy, velocityField.data))

In [94]:
test2(True)
test2(False)

Isoviscous:  True
Vel field unchanged:  True
Isoviscous:  False
Vel field unchanged:  True


In [61]:
test(True)
test(False)

Isoviscous:  True
318923395.94400495
True
318923395.94400495
True
318923395.94400495
True
318923395.94400495
True
318923395.94400495
True
318923395.94400495
True
318923395.94400495
True
318923395.94400495
True
318923395.94400495
True
318923395.94400495
True
Isoviscous:  False
20811.903859400623
False
8701.866494750899
False
7337.442936742046
False
7043.7919503240455
False
6969.741030257115
False
6948.663774924843
False
6941.753808488464
False
6939.207006151775
False
6938.136001709274
False
6937.619020246664
False
